In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

2024-09-12 17:02:52.713115: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-12 17:02:52.715624: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-12 17:02:52.724787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 17:02:52.739997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 17:02:52.744592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 17:02:52.755220: I tensorflow/core/platform/cpu_feature_gu

In [11]:
import os
os.getcwd()
concat = pd.read_csv('../data/base_dataset_7.csv')
concat.shape

(1128094, 27)

In [12]:
str_columns = ['본번', '부번', '거래취소여부', '구', '동', '강남여부', '신축여부']

concat[str_columns] = concat[str_columns].astype(str)

# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'is_test', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수']
범주형 변수: ['번지', '본번', '부번', '아파트명', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부']


In [13]:
# 범주형 변수에 대한 보간
concat[categorical_columns] = concat[categorical_columns].fillna('NULL')

# 연속형 변수에 대한 보간 (선형 보간)
concat[continuous_columns] = concat[continuous_columns].interpolate(method='linear', axis=0)

print(concat.isnull().sum())         # 결측치가 보간된 모습을 확인해봅니다.
print(concat.shape)

번지         0
본번         0
부번         0
아파트명       0
전용면적       0
계약일        0
층          0
건축년도       0
target     0
is_test    0
주소         0
x          0
y          0
계약년        0
계약월        0
거래취소여부     0
거래일건물연식    0
구          0
동          0
강남여부       0
신축여부       0
건축면적       0
연면적        0
대지면적       0
건폐율        0
용적율        0
평균층수       0
dtype: int64
(1128094, 27)


In [14]:
# test와 train 분리
df = concat.query('is_test == 0')  
df_test = concat.query('is_test == 1')  

# 전용면적 375 이상인 것들은 제거
df= df[df['전용면적'] < 375]

# 층 음수 처리
df['층'] = df['층'].apply(lambda x: 1 if x < 0 else x)
df_test['층'] = df_test['층'].apply(lambda x: 1 if x < 0 else x)

# 건축년도 1965년도 이후에 지어진것만 처리
df= df[df['건축년도'] > 1965]

# 거래일 건물 연식 음수 처리
df= df[df['거래일건물연식'] > 0]
concat_select = pd.concat([df, df_test])
concat_select.info()       # 최종 데이터셋은 아래와 같습니다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1108303 entries, 0 to 1128093
Data columns (total 27 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   번지       1108303 non-null  object 
 1   본번       1108303 non-null  object 
 2   부번       1108303 non-null  object 
 3   아파트명     1108303 non-null  object 
 4   전용면적     1108303 non-null  float64
 5   계약일      1108303 non-null  int64  
 6   층        1108303 non-null  int64  
 7   건축년도     1108303 non-null  int64  
 8   target   1108303 non-null  float64
 9   is_test  1108303 non-null  int64  
 10  주소       1108303 non-null  object 
 11  x        1108303 non-null  float64
 12  y        1108303 non-null  float64
 13  계약년      1108303 non-null  int64  
 14  계약월      1108303 non-null  int64  
 15  거래취소여부   1108303 non-null  object 
 16  거래일건물연식  1108303 non-null  int64  
 17  구        1108303 non-null  object 
 18  동        1108303 non-null  object 
 19  강남여부     1108303 non-null  object 
 20  신축

In [15]:
from math import radians, sin, cos, sqrt, atan2

# 하버사인 공식으로 두 지리적 좌표 간의 거리를 계산하는 함수
def haversine_np(lon1, lat1, lon2, lat2):
    R = 6371  # 지구 반경 (킬로미터)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance
subway_path = pd.read_csv('../data/subway_feature.csv')
bus_path = pd.read_csv('../data/bus_feature.csv')
# concat_select 좌표와 subway_path 좌표를 numpy 배열로 변환
concat_coords = np.array([concat_select['x'], concat_select['y']]).T
subway_coords = np.array([subway_path['경도'], subway_path['위도']]).T
bus_coords = np.array([bus_path['X좌표'], bus_path['Y좌표']]).T
# 빈 리스트를 만들어서 최단 거리를 저장할 것임
shortest_distances_subway = []
shortest_distances_bus = []
# 벡터 연산을 사용하여 가장 가까운 지하철 거리 찾기
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], subway_coords[:, 0], subway_coords[:, 1])
    min_distance = np.min(distances)
    shortest_distances_subway.append(min_distance)
# 벡터 연산을 사용하여 가장 가까운 버스 거리 찾기
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], bus_coords[:, 0], bus_coords[:, 1])
    min_distance = np.min(distances)
    shortest_distances_bus.append(min_distance)
# concat_select에 최단 거리를 추가
concat_select['지하철최단거리'] = shortest_distances_subway
concat_select['버스최단거리'] = shortest_distances_bus
concat_select['버스최단거리'].describe()

#거리별 가중치 부여
concat_select['지하철최단거리'] = concat_select['지하철최단거리'].apply(lambda x: 3 if 0 < x < 0.3 else (2 if 0.3 < x < 0.65 else(1 if 0.65<x<1.0 else 0) ))
concat_select['버스최단거리'] = concat_select['버스최단거리'].apply(lambda x: 2 if 0 < x < 0.5 else (1 if 0.5 < x < 1 else 0))


#10분 이내로 

  0%|          | 0/1108303 [00:00<?, ?it/s]

100%|██████████| 1108303/1108303 [11:07<00:00, 1659.76it/s]


In [16]:
brand_patterns = {
   #1군 하이엔트
   '디에이치':3, '푸르지오써밋': 3, '르엘':3,'오티에르':3, '아크로':3,
   #1군
   '힐스테이트': 2, '푸르지오':2, '롯데캐슬':2, '더샵':2, 'e-편한세상':2,'래미안':2 , '아이파크':2, '자이':2,
   #2군 하이엔드
   '위브더제니스':2,'호반써밋':2,
    #2군
    '위브': 1,'우미린':1,'한화포레나':1, '서희스타힐스':1,'더플래티넘':1, '한라비발디':1,'호반베르디움':1, '데시앙':1, '센트레빌':1,'SKVIEW':1,''
    '하늘채': 1,'스위첸':1
}
# 특정 문자열을 포함하는 경우에 따라 값을 반환하는 함수 정의
def assign_brand_value(apartment_name):
    for pattern, value in brand_patterns.items():
        if pattern in apartment_name:
            return value
    return 0
# '아파트브랜드이름'이라는 새로운 열 생성
concat_select['아파트브랜드이름'] = concat_select['아파트명'].apply(assign_brand_value)
print(concat_select.shape, concat_select.columns)

(1108303, 30) Index(['번지', '본번', '부번', '아파트명', '전용면적', '계약일', '층', '건축년도', 'target',
       'is_test', '주소', 'x', 'y', '계약년', '계약월', '거래취소여부', '거래일건물연식', '구', '동',
       '강남여부', '신축여부', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리',
       '버스최단거리', '아파트브랜드이름'],
      dtype='object')


In [99]:
# # 금리 데이터 로드 및 처리
# interest = pd.read_csv('/Users/minseok/Downloads/xgboost/data/한국기준금리07-23.csv')

# interest['계약년월'] = interest['날짜'].astype(str).str[:6]

# interest_grouped = interest.groupby('계약년월').agg({'기준금리': 'mean'}).reset_index()

# concat_select['계약년월'] = concat_select['계약년'].astype(str) + concat_select['계약월'].astype(str)

# concat_select = pd.merge(concat_select, interest_grouped, on='계약년월', how='left')



# print(concat_select.shape, concat_select.columns)

In [100]:
# concat_select = concat_select.drop(columns=['계약년월'])
# print(concat_select.shape, concat_select.columns)

In [20]:
import os
os.getcwd()

'/data/ephemeral/home/code'

In [22]:
school_path = pd.read_csv('./data/청주대학교_지방교육재정연구원_초중등학교위치_20240322.csv', encoding='EUC-KR')
h_school = school_path[school_path['학교급구분'] == '고등학교']
m_school = school_path[school_path['학교급구분'] == '중학교']
p_school = school_path[school_path['학교급구분'] == '초등학교']
h_school_coords = np.array([h_school['경도'], h_school['위도']]).T
m_school_coords = np.array([m_school['경도'], m_school['위도']]).T
p_school_coords = np.array([p_school['경도'], p_school['위도']]).T
# 빈 리스트를 만들어서 최단 거리를 저장할 것임
shortest_distances_h_school = []
shortest_distances_m_school = []
shortest_distances_p_school = []
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], h_school_coords[:, 0], h_school_coords[:, 1])
    min_distance1 = np.min(distances)
    shortest_distances_h_school.append(min_distance1)
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], m_school_coords[:, 0], m_school_coords[:, 1])
    min_distance2 = np.min(distances)
    shortest_distances_m_school.append(min_distance2)
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], p_school_coords[:, 0], p_school_coords[:, 1])
    min_distance3 = np.min(distances)
    shortest_distances_p_school.append(min_distance3)
concat_select['고등학교최단거리'] = shortest_distances_h_school
concat_select['중등학교최단거리'] = shortest_distances_m_school
concat_select['초등학교최단거리'] = shortest_distances_p_school
concat_select['초등학교최단거리'] = concat_select['초등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select['중등학교최단거리'] = concat_select['중등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select['고등학교최단거리'] = concat_select['고등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select.head()

100%|██████████| 1108303/1108303 [05:53<00:00, 3137.84it/s]


,번지,본번,부번,아파트명,전용면적,계약일,층,건축년도,target,is_test,...,대지면적,건폐율,용적율,평균층수,지하철최단거리,버스최단거리,아파트브랜드이름,고등학교최단거리,중등학교최단거리,초등학교최단거리
0,658-1,658,1,개포6차우성,79.97,8,3,1987,124000.0,0,...,171357.334166,297.555783,3143.83423,4.90097,0,2,0,3,1,3
1,658-1,658,1,개포6차우성,79.97,22,4,1987,123500.0,0,...,171357.334166,297.555783,3143.83423,4.90097,0,2,0,3,1,3
2,658-1,658,1,개포6차우성,54.98,28,5,1987,91500.0,0,...,171357.334166,297.555783,3143.83423,4.90097,0,2,0,3,1,3
3,658-1,658,1,개포6차우성,79.97,3,4,1987,130000.0,0,...,171357.334166,297.555783,3143.83423,4.90097,0,2,0,3,1,3
4,658-1,658,1,개포6차우성,79.97,8,2,1987,117000.0,0,...,171357.334166,297.555783,3143.83423,4.90097,0,2,0,3,1,3


In [95]:
# str_columns = ['초등학교최단거리', '중등학교최단거리', '고등학교최단거리']

# concat_select[str_columns] = concat_select[str_columns].astype(str)

# print(concat_select.shape)

In [23]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_select.query('is_test==0')
dt_test = concat_select.query('is_test==1')

# 결과 출력
print(dt_train.shape, dt_test.shape)

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
dt_test['target'] = 0

(1099031, 33) (9272, 33)
(1099031, 32) (9272, 32)


In [24]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리', '버스최단거리', '아파트브랜드이름', '고등학교최단거리', '중등학교최단거리', '초등학교최단거리']
범주형 변수: ['번지', '본번', '부번', '아파트명', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부']


100%|██████████| 10/10 [00:02<00:00,  3.38it/s]


In [25]:
dt_train = pd.get_dummies(dt_train, columns=['구'])
dt_test = pd.get_dummies(dt_test, columns=['구'])

In [26]:
columns_to_delete = ['번지', '부번', '본번']

# # 열 삭제
dt_train = dt_train.drop(columns=columns_to_delete)
dt_test = dt_test.drop(columns=columns_to_delete)

In [27]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [28]:
# #  XGBoost 사용

# import xgboost as xgb

# model = xgb.XGBRegressor(objective='reg:squarederror', 
#                          n_estimators=2000,  
#                          learning_rate=0.1,  
#                          max_depth=10,
#                          random_state=1)

# model.fit(X_train, y_train)

# pred_2 = model.predict(X_val)

# # 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
# print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred_2))}')

RMSE test: 5571.655077581897


In [32]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
import joblib
import os

# RMSE 계산 함수
def RMSE(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [9000, 10000, 11000, 12000],  # 더 높은 n_estimators 설정
    'learning_rate': [0.05, 0.1, 0.15],  # 학습률을 더 세밀하게 설정
    'max_depth': [8, 9, 10],  # max_depth 범위를 확장하여 조정
    'min_child_weight': [1, 3],  # 현재 최적값 유지
    'subsample': [0.8],  # 그대로 유지
    'colsample_bytree': [0.8],  # 그대로 유지
    'gamma': [0],  # 그대로 유지
    'lambda': [1],  # 그대로 유지
    'alpha': [0]  # 그대로 유지
}

# 체크포인트 경로 설정
checkpoint_path = "grid_search_checkpoint.pkl"

# 체크포인트에서 학습 재개
if os.path.exists(checkpoint_path):
    start_idx, best_params, best_rmse = joblib.load(checkpoint_path)
    print(f"Checkpoint 로드 완료, {start_idx}번째 파라미터부터 재개.")
else:
    start_idx = 0
    best_params = None
    best_rmse = float('inf')
    print("새로운 학습 시작.")

# XGBRegressor 모델 정의
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=2024
)

# 파라미터 그리드 순회
for idx, params in enumerate(ParameterGrid(param_grid)):
    if idx < start_idx:
        continue  # 이전에 완료된 파라미터 조합은 건너뜀
    
    # 모델 학습
    print(f"{idx + 1}번째 파라미터 조합 학습 중: {params}")
    xgb_model.set_params(**params)
    xgb_model.fit(X_train, y_train)

    # 검증 세트에서 예측
    pred_val = xgb_model.predict(X_val)
    val_rmse = RMSE(y_val, pred_val)
    print(f"Validation RMSE: {val_rmse:.4f}")

    # 최적 파라미터 갱신
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_params = params

    # 진행 상황을 체크포인트에 저장
    joblib.dump((idx + 1, best_params, best_rmse), checkpoint_path)
    print(f"Checkpoint 저장됨: {checkpoint_path}")

# 최적 파라미터 및 성능 출력
print(f"최적 파라미터: {best_params}, Best RMSE: {best_rmse:.4f}")

# 최적 모델로 학습 후 모델 저장
best_xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)
best_xgb_model.fit(X_train, y_train)
best_xgb_model.save_model(f"xgb_best_model_rmse_{best_rmse:.4f}.model")

# 체크포인트 삭제
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)
    print("Checkpoint 파일 삭제 완료.")

새로운 학습 시작.
1번째 파라미터 조합 학습 중: {'alpha': 0, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 9000, 'subsample': 0.8}
Validation RMSE: 5444.0422
Checkpoint 저장됨: grid_search_checkpoint.pkl
2번째 파라미터 조합 학습 중: {'alpha': 0, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 10000, 'subsample': 0.8}
Validation RMSE: 5444.7013
Checkpoint 저장됨: grid_search_checkpoint.pkl
3번째 파라미터 조합 학습 중: {'alpha': 0, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 11000, 'subsample': 0.8}
Validation RMSE: 5444.6754
Checkpoint 저장됨: grid_search_checkpoint.pkl
4번째 파라미터 조합 학습 중: {'alpha': 0, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 12000, 'subsample': 0.8}
Validation RMSE: 5445.3693
Checkpoint 저장됨: g

In [37]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import os
import sys

# RMSE 계산 함수
def RMSE(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# 파일에 로그 저장
log_filename = "training_validation_results_randomsearch.txt"

# 파일에 결과를 기록하는 함수
def log_results(filename, message):
    with open(filename, 'a') as f:
        f.write(f"{message}\n")
        f.write("="*50 + "\n")  # 구분선 추가

# 학습 로그를 파일에 기록하는 함수
class FileLogger:
    def __init__(self, file_name):
        self.file_name = file_name
    
    def __enter__(self):
        self.log_file = open(self.file_name, 'a')
        self.original_stdout = sys.stdout
        sys.stdout = self.log_file  # 출력을 파일로 리다이렉트
        return self.log_file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self.original_stdout  # 원래 stdout으로 복구
        self.log_file.close()

# XGBRegressor 모델 정의
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=2024)

# 하이퍼파라미터 범위 설정
param_distributions = {
    'n_estimators': np.arange(2000, 10000, 500),  # 트리 개수의 더 넓은 범위
    'learning_rate': np.linspace(0.01, 0.02, 10),  # 다양한 학습률
    'max_depth': np.arange(3, 10, 1),  # 깊이 범위 확장
    'min_child_weight': np.arange(1, 10, 1),  # 리프 노드 최소 가중치
    'subsample': np.linspace(0.5, 0.9, 5),  # 샘플링 비율
    'colsample_bytree': np.linspace(0.7, 0.9, 3),  # 피처 샘플링 비율
    'gamma': np.linspace(0, 2, 5),  # 손실 감소를 위한 노드 분할 기준
    'lambda': np.linspace(1, 10, 5),  # L2 정규화 가중치
    'alpha': np.linspace(0, 5, 5)  # L1 정규화 가중치
}

# RandomizedSearchCV 수정
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=50,  # 50번의 무작위 탐색
    scoring='neg_root_mean_squared_error',  # RMSE 최적화
    cv=5,  # 5-fold 교차 검증
    verbose=3,  # 상세한 로그 출력
    n_jobs=-1,
    random_state=2024
)

# 학습 로그를 파일로 기록
with FileLogger(log_filename):
    print("RandomizedSearchCV 시작")

    # RandomizedSearch 실행
    random_search.fit(X_train, y_train)

    # 각 iteration 및 fold 결과를 로그 파일에 저장
    results = random_search.cv_results_

    for i in range(len(results['params'])):
        log_results(log_filename, f"Iteration {i + 1} 결과:")
        log_results(log_filename, f"Parameters: {results['params'][i]}")
        for fold in range(random_search.cv):
            log_results(log_filename, f"Fold {fold + 1} Score: {results['split{}_test_score'.format(fold)][i]:.4f}")

# 최적 하이퍼파라미터 출력 및 저장
best_params = random_search.best_params_
best_rmse = -random_search.best_score_

# 로그 파일에 저장
log_results(log_filename, f"Best Parameters: {best_params}")
log_results(log_filename, f"Best RMSE: {best_rmse:.4f}")

# 최적 하이퍼파라미터로 XGBoost 모델 학습 전에 로그 파일에 기록
log_results(log_filename, f"Training XGBoost with best parameters: {best_params}")

# 최적 하이퍼파라미터로 XGBoost 모델 학습
best_xgb_model = xgb.XGBRegressor(
    **best_params, 
    objective='reg:squarederror',
    random_state=2024
)

# 학습 로그를 파일로 기록
with FileLogger(log_filename):
    print(f"XGBoost 학습 시작 - n_estimators: {best_params['n_estimators']}")
    
    # 최적 모델로 학습
    best_xgb_model.fit(X_train, y_train)

# 검증 세트에서 예측
pred_val = best_xgb_model.predict(X_val)
pred_train = best_xgb_model.predict(X_train)

# RMSE 계산 (학습 데이터와 검증 데이터)
train_rmse = RMSE(y_train, pred_train)
val_rmse = RMSE(y_val, pred_val)

# 파라미터 및 성능을 반영한 파일명 생성
model_filename = (f"xgb_n{best_params['n_estimators']}_lr{best_params['learning_rate']}_md{best_params['max_depth']}_"
                  f"mcw{best_params['min_child_weight']}_sub{best_params['subsample']}_csbt{best_params['colsample_bytree']}_"
                  f"g{best_params['gamma']}_lambda{best_params['lambda']}_alpha{best_params['alpha']}_"
                  f"train_rmse{train_rmse:.4f}_val_rmse{val_rmse:.4f}.model")

# 모델 저장
best_xgb_model.save_model(model_filename)

# 결과 로그 파일에 저장
result_info = (
    f"Model Training End\n"
    f"Model: {model_filename}\n"
    f"Train RMSE: {train_rmse:.4f}, Validation RMSE: {val_rmse:.4f}"
)
log_results(log_filename, result_info)

print(f"모델 저장됨: {model_filename}, Train RMSE: {train_rmse:.4f}, Validation RMSE: {val_rmse:.4f}")

[CV 5/5] END alpha=5.0, colsample_bytree=0.7, gamma=2.0, lambda=7.75, learning_rate=0.015555555555555555, max_depth=3, min_child_weight=5, n_estimators=3500, subsample=0.6;, score=-11888.582 total time= 3.1min
[CV 2/5] END alpha=5.0, colsample_bytree=0.7, gamma=2.0, lambda=7.75, learning_rate=0.015555555555555555, max_depth=3, min_child_weight=5, n_estimators=3500, subsample=0.6;, score=-11586.441 total time= 3.1min
[CV 1/5] END alpha=5.0, colsample_bytree=0.7, gamma=2.0, lambda=7.75, learning_rate=0.015555555555555555, max_depth=3, min_child_weight=5, n_estimators=3500, subsample=0.6;, score=-11491.662 total time= 3.1min
[CV 4/5] END alpha=5.0, colsample_bytree=0.7, gamma=2.0, lambda=7.75, learning_rate=0.015555555555555555, max_depth=3, min_child_weight=5, n_estimators=3500, subsample=0.6;, score=-11504.761 total time= 3.1min
[CV 3/5] END alpha=5.0, colsample_bytree=0.7, gamma=2.0, lambda=7.75, learning_rate=0.015555555555555555, max_depth=3, min_child_weight=5, n_estimators=3500, su

In [38]:
from sklearn import metrics

# 예측 수행
pred_2 = best_xgb_model.predict(X_val)

# RMSE 계산
rmse_result = np.sqrt(metrics.mean_squared_error(y_val, pred_2))

# 결과 출력 및 파일 저장
print(f'RMSE test: {rmse_result}')

# 결과를 파일에 저장
with open("rmse_result.txt", "w") as f:
    f.write(f'RMSE test: {rmse_result}\n')
    f.write(f"최적 파라미터: {best_params}, Best RMSE: {best_rmse:.4f}")

RMSE test: 5447.239595048805


In [41]:
# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open('saved_modelrandom.pkl', 'wb') as f:
    pickle.dump(best_xgb_model, f)

In [42]:
# 저장된 모델을 불러옵니다.
with open('saved_modelrandom.pkl', 'rb') as f:
    model = pickle.load(f)

In [43]:
X_test = dt_test.drop(['target'], axis=1)

real_test_pred = model.predict(X_test)

In [44]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output_random.csv', index=False)